<a href="https://colab.research.google.com/github/SMBH-1/tbd/blob/main/ML_Project_Iris_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd

In [2]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']


train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
#Using keras (library that serves as interface for TensorFlow) to grab datasets and read them into a pandas df


train_y = train.pop('Species')
test_y = test.pop('Species')
train.head() #Species col in training data is now removed

573/573 [==============================] - 0s 0us/step


,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [3]:
def input_fn(features, labels, training=True, batch_size=256):
  #Convert inputs to a dataset; notice no epochs like in regression problem
  dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))

  #Shuffle & repeat if in training mode
  if training:
    dataset = dataset.shuffle(1000).repeat()
  
  return dataset.batch(batch_size)

#Feature columns describe how to use the input
my_feature_columns = []
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [ ]:
#Build a DNN w/ 2 hidden layers w/ 30 and 10 hidden nodes each.

classifier = tf.estimator.DNNClassifier( #estimator module stores premade tf models
    feature_columns = my_feature_columns,
    #Two hidden layers of 30 & 10 nodes respectively
    hidden_units = [30,10],
    #Model must choose between 3 classes.
    n_classes=3)

#Train:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000
)

In [5]:
#Evaluate the Model:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False)
)
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


Test set accuracy: 0.933



In [6]:
def input_fn(features, batch_size=256):
  #Convert the inputs to a dataset w/o labels
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted.")

for feature in features:
  valid = True
  while valid:
    val = input(feature + ":")
    if not val.isdigit(): valid = False
  
  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
  class_id = pred_dict['class_ids'][0]
  probability = pred_dict['probabilities'][class_id]

  print(pred_dict)

  print('Prediction is "{}" ({:.1f}%)'.format(
      SPECIES[class_id], 100*probability)
  )

expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1]
}

Please type numeric values as prompted.
SepalLength:5.1
SepalWidth:3.3
PetalLength:1.7
PetalWidth:0.5
{'logits': array([ 2.2729828 ,  0.99598676, -2.5377748 ], dtype=float32), 'probabilities': array([0.77699155, 0.21668254, 0.00632602], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Setosa" (77.7%)
